# Building  Text-to-SQL capability to Amazon Athena using Amazon Bedrock




## Contents

1. [Objective](#Objective)
1. [Background](#Background-(Problem-Description-and-Approach))
1. [Overall Workflow](#Overall-Workflow)
1. [Conclusion](#Conclusion)


## Objective

This notebook shows is to provide the code snippets within an executable flow from [this AWS Blog post](https://aws.amazon.com/blogs/machine-learning/build-a-robust-text-to-sql-solution-generating-complex-queries-self-correcting-and-querying-diverse-data-sources/).

## Background (Problem Description and Approach)

- **Problem statement**: 

Text-to-SQL solutions aim to generate SQL queries from natural language to enable non-technical users to access and analyze data. However, existing solutions face challenges related to ambiguity in natural language, needing to recreate capabilities for different databases, and collecting comprehensive metadata. The proposed solution in the text aims to address these challenges by incorporating metadata from AWS Glue Data Catalog, evaluating and correcting generated SQL queries using Amazon Athena feedback with multi-pass prompting, and leveraging Athena's support for diverse data sources.

- **Our approach**: 

[`RAG`] The [RAG approach](https://aws.amazon.com/what-is/retrieval-augmented-generation/) offers several advantages. First, it gives up-to-date, precise responses. Rather than relying only on fixed, outdated training data, RAG utilizes current external sources to formulate its answers. In this solution, we used RAG to increase the accuracy of table name from [AWS Glue Data Catalog](https://docs.aws.amazon.com/glue/latest/dg/catalog-and-crawler.html). 

[`Vector Store`] [Amazon OpenSearch](https://aws.amazon.com/opensearch-service/) offers three vector engines to choose from, each catering to different use cases.Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. This code bases used [FAISS for similiarity search](https://aws.amazon.com/about-aws/whats-new/2023/10/amazon-opensearch-service-vector-query-filters-faiss/).

[`Amazon Athena`] [Amazon Athena](https://aws.amazon.com/athena/) is a serverless, interactive analytics service built on open-source frameworks, supporting open-table and file formats. Athena provides a simplified, flexible way to analyze petabytes of data where it lives. In this solution, we used Amazon Athena as the SQL engine to 

[`Amazon Bedrock`] [Amazon Bedrock](https://aws.amazon.com/bedrock/) is a fully managed service that offers a choice of high-performing foundation models (FMs) from leading AI companies like AI21 Labs, Anthropic, Cohere, Meta, Stability AI, and Amazon via a single API, along with a broad set of capabilities you need to build generative AI applications with security, privacy, and responsible AI.

We used Bedrock with multi-step / multi-pass component which allows the LLM to correct the generated SQL query for accuracy. Here, the generated SQL is sent for syntax errors. We use Athena error messages to enrich our prompt for the LLM for more accurate and effective corrections in the generated SQL.

- *[RAG on AWS](https://aws.amazon.com/what-is/retrieval-augmented-generation/)*
- *[The langchain OpenSearch documentation](https://python.langchain.com/en/latest/ecosystem/opensearch.html)*
- *[Amazon OpenSearch service documentation](https://docs.aws.amazon.com/opensearch-service/index.html)*
- *[Amazon OpenSearch supports efficient vector](https://aws.amazon.com/about-aws/whats-new/2023/10/amazon-opensearch-service-vector-query-filters-faiss/)*



---

## Overall Workflow

**Prerequisite**

The following are prerequisites that needs to be accomplised before executing this notebook.
This Notebook can be executed via a Sagemaker instance or via a VS Code editor
- Create a role having access to bedrock, glue,athena, s3,lakeformation. 
- Assign the role to the Sagemaker instance or to the instance where VS Code editor is running
- Glue Database and tables. Provided spark notebook to create.
- An Amazon OpenSearch cluster for storing embeddings.Here Opensearch credenitals are in notebooks. However Opensearch cluster's access credentials (username and password) can be stored in AWS Secrets Mananger by following steps described [here](https://docs.aws.amazon.com/secretsmanager/latest/userguide/managing-secrets.html).

**The  workflow for this notebook is as follows:**
*Please read [Readme.md](https://github.com/aws-samples/text-to-sql-for-athena/blob/claude3branch/README.md) to learn about the detailed steps*

##### Step 1: Setup
Install the required packages.

In [16]:
# !pip3 install boto3
# !pip3 install jq
# !pip3 install langchain
# !pip3 install langchain-community langchain-core
# !pip3 install pandas
# !pip3 install opensearch-py
# !pip3 install langchain-aws
# !pip3 install requests-aws4auth
# !pip3 install botocore

##### Step 2: Import all modules. There are some modules in other folder.

In [17]:
import boto3
from botocore.config import Config
from langchain_community.embeddings import BedrockEmbeddings
from langchain_aws import BedrockLLM
import traceback
import logging
import json
import os,sys
import re
import time
import pandas as pd
import io
from boto_client import Clientmodules
from llm_basemodel import LanguageModel
from athena_execution import AthenaQueryExecute
from openSearchVCEmbedding import EmbeddingBedrockOpenSearch

##### Step 3: Checking access to Bedrock

In [18]:
session = boto3.session.Session()
bedrock_client = session.client('bedrock')
print(bedrock_client.list_foundation_models()['modelSummaries'][0])

{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large', 'modelId': 'amazon.titan-tg1-large', 'modelName': 'Titan Text Large', 'providerName': 'Amazon', 'inputModalities': ['TEXT'], 'outputModalities': ['TEXT'], 'responseStreamingSupported': True, 'customizationsSupported': [], 'inferenceTypesSupported': ['ON_DEMAND'], 'modelLifecycle': {'status': 'ACTIVE'}}


##### Step 4: Invoking Athena and Bedrock

In [19]:
rqstath = AthenaQueryExecute()

athena client created 
s3 client created !!


In [20]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [21]:
index_name = 'text_to_sql_index'
domain = 'https://e0hc00i67ga6mpn1xkxa.us-east-1.aoss.amazonaws.com' ##-- update here with your OpenSearch domain
region = 'us-east-1'
vector_name = 'embeddings_vector'
fieldname = 'id'

In [22]:
ebropen2 = EmbeddingBedrockOpenSearch(domain,  vector_name,  fieldname)
if ebropen2 is None:
    print("ebropen2 is null")
else:
    attrs = vars(ebropen2)
    print(', '.join("%s: %s" % item for item in attrs.items()))

bedrock runtime client created 


bedrockllm
bedrock_client: <botocore.client.BedrockRuntime object at 0x000001B951439210>, language_model: <llm_basemodel.LanguageModel object at 0x000001B95143AD40>, llm: client=<botocore.client.BedrockRuntime object at 0x000001B951439210> model_id='anthropic.claude-v2:1' model_kwargs={'temperature': 0, 'top_k': 20, 'top_p': 1, 'stop_sequences': ['\n\nHuman:']}, embeddings: client=<botocore.client.BedrockRuntime object at 0x000001B951439210> region_name=None credentials_profile_name=None model_id='amazon.titan-embed-text-v2:0' model_kwargs=None endpoint_url=None normalize=False, opensearch_domain_endpoint: https://e0hc00i67ga6mpn1xkxa.us-east-1.aoss.amazonaws.com, http_auth: <requests_aws4auth.aws4auth.AWS4Auth object at 0x000001B950DFE200>, vector_name: embeddings_vector, fieldname: id


##### Step 5: Core logic
1. getEmbeddding : Take the input user query and vector search to find the schema from vector db created.
2. generate_sql: Taking the input prompt, generate sql . syntax_checker helps to check the sql syntax.


In [23]:
class RequestQueryBedrock:
    def __init__(self, ebropen2):
        self.ebropen2 = ebropen2
        self.bedrock_client = ebropen2.bedrock_client
        if self.bedrock_client is None:
            self.bedrock_client = Clientmodules.createBedrockRuntimeClient()
        else:
            print("the bedrock_client is not null")
        self.language_model = LanguageModel(self.bedrock_client)
        self.llm = self.language_model.llm

    def getOpenSearchEmbedding(self, index_name, user_query):
        vcindxdoc = self.ebropen2.getDocumentfromIndex(index_name=index_name)
        document = self.ebropen2.getSimilaritySearch(user_query, vcindxdoc)
        return self.ebropen2.get_data(document)

    def generate_sql(self, prompt, max_attempt=4) -> str:
        attempt = 0
        error_messages = []
        prompts = [prompt]
        sql_query = ""

        while attempt < max_attempt:
            logger.info(f'Sql Generation attempt Count: {attempt + 1}')
            try:
                logger.info(f'we are in Try block to generate the sql and count is :{attempt + 1}')
                generated_sql = self.llm.invoke(prompt)

                # Handle AIMessage object
                if isinstance(generated_sql, AIMessage):
                    content = generated_sql.content
                elif isinstance(generated_sql, str):
                    content = generated_sql
                else:
                    content = str(generated_sql)

                # Extract SQL query from content
                sql_parts = content.split("```")
                if len(sql_parts) > 1:
                    query_str = sql_parts[1]
                else:
                    query_str = content

                query_str = " ".join(query_str.split("\n")).strip()
                sql_query = query_str[3:] if query_str.lower().startswith("sql") else query_str

                print(sql_query)
                syntaxcheckmsg = rqstath.syntax_checker(sql_query)
                if syntaxcheckmsg == 'Passed':
                    logger.info(f'syntax checked for query passed in attempt number :{attempt + 1}')
                    return sql_query
                else:
                    prompt = f"""{prompt}
                        This is syntax error: {syntaxcheckmsg}.
                        To correct this, please generate an alternative SQL query which will correct the syntax error.
                        The updated query should take care of all the syntax issues encountered.
                        Follow the instructions mentioned above to remediate the error.
                        Update the below SQL query to resolve the issue:
                        {sql_query}
                        Make sure the updated SQL query aligns with the requirements provided in the initial question."""
                    prompts.append(prompt)
            except Exception as e:
                print(e)
                logger.error('FAILED')
                msg = str(e)
                error_messages.append(msg)
            finally:
                attempt += 1

        # If all attempts fail, raise an exception with details
        raise Exception(f"Failed to generate SQL after {max_attempt} attempts. Errors: {', '.join(error_messages)}")

##### Create an instance of  RequestQueryBedrock class

In [24]:
rqst = RequestQueryBedrock(ebropen2)

the bedrock_client is not null
bedrockllm


In [25]:
def userinput(user_query):
    logger.info(f'Searching metadata from vector store')
    vector_search_match = rqst.getOpenSearchEmbedding(index_name, user_query)

    details = f"""It is important that the SQL query complies with Athena syntax.

              For our testing purposes, please only use the following data source(s), database(s) and table(s):

              Data Source: AWSDataCatalog
              Database: imdb_stg
              Tables: basics, ratings

              During a join, if two column names are the same please use alias (example: basics.tconst in select
              statement). It is also important to pay attention to and not alter column format: if a column is string,
              then leave column formatting alone and return a value that is a string.

              If you are writing CTEs then include all the required columns. While concatenating a non-string column,
              make sure to cast the column to string format first. If you encounter any instances where we must
              compare date columns to strings, please cast the string input as a date and format as such.

              REMEMBER: Only use the data source(S), database(s), and table(s) mentioned above. In addition,
              always include the database name along with the table name in the query."""

    final_question = "\n\nHuman:" + details + vector_search_match + user_query + "\n\nAssistant:"
    print("FINAL QUESTION :::" + final_question)

    try:
        answer = rqst.generate_sql(final_question)
        return answer
    except Exception as e:
        logger.error(f"Failed to generate SQL: {str(e)}")
        return None  # Or handle the error in a way that makes sense for your application

##### Step 6: User input in Natural Language

In [26]:
user_query = 'How many records in our database are from the year 1892?'

In [27]:
querygenerated = userinput(user_query)

Searching metadata from vector store
Searching metadata from vector store


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\python310\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "D:\Users\jcarhart-health\AppData\Local\Temp\1\ipykernel_8180\3239037324.py", line 1, in <module>
    querygenerated = userinput(user_query)
  File "D:\Users\jcarhart-health\AppData\Local\Temp\1\ipykernel_8180\2183822863.py", line 3, in userinput
    vector_search_match = rqst.getOpenSearchEmbedding(index_name, user_query)
  File "D:\Users\jcarhart-health\AppData\Local\Temp\1\ipykernel_8180\3729880947.py", line 14, in getOpenSearchEmbedding
    document = self.ebropen2.getSimilaritySearch(user_query, vcindxdoc)
  File "C:\code\text-to-sql-for-athena\openSearchVCEmbedding.py", line 138, in getSimilaritySearch
  File "C:\python310\lib\site-packages\langchain_community\vectorstores\opensearch_vector_search.py", line 818, in similarity_search
    docs_with_scores = self.similarity_search_with_score(


##### Step 7: Sql Query and Query Execution output

In [28]:
import pprint
my_printer = pprint.PrettyPrinter()
my_printer.pprint(querygenerated)

NameError: name 'querygenerated' is not defined

In [ ]:
QueryOutput = rqstath.execute_query(querygenerated)

In [ ]:
print(QueryOutput)


##### Cleanup

To avoid incurring future charges, delete the resources.


##### Conclusion
In this notebook we were able to see how to use bedrock to deploy LLM Model to generate embeddings,then ingest those embeddings into OpenSearch and finally do a similarity search for user input to the documents (embeddings) stored in OpenSearch. Please read our [AWS Blog Post](https://aws.amazon.com/blogs/machine-learning/build-a-robust-text-to-sql-solution-generating-complex-queries-self-correcting-and-querying-diverse-data-sources/) on this topic to learn more about the solution.
